In [1]:
from dotenv import load_dotenv
import os
from web3 import Web3
from eth_account import Account
import pandas as pd
import time
import random
import math
from diskcache import Cache
import datetime as dt
from datetime import timedelta
from uniswap import Uniswap
import json
from pydantic import BaseModel, Field
from typing import List
import numpy as np
from openai import OpenAI
import requests
import plotly.express as px

load_dotenv()

True

In [2]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
flipside_api_key = os.getenv("FLIPSIDE_API_KEY")
FRED_API_KEY = os.getenv('FRED_API_KEY')

ACCOUNT_ADDRESS = os.getenv('ACCOUNT_ADDRESS')
PRIVATE_KEY = os.getenv('PRIVATE_KEY')
GATEWAY = os.getenv('ARBITRUM_GATEWAY')

In [3]:
os.chdir('..')

In [4]:
cache = Cache('classifier_data')

In [5]:
historical_data = cache.get(f'historical_data', pd.DataFrame())
historical_port_values = cache.get(f'historical_port_values', pd.DataFrame())
oracle_prices = cache.get(f'oracle_prices',pd.DataFrame())
last_rebalance_time = cache.get(f'last_rebalance_time', None)
model_actions = cache.get(f'actions', pd.DataFrame()) 

In [6]:
from python_scripts.web3_utils import *
from python_scripts.apis import token_classifier_portfolio, flipside_api_results
from python_scripts.macro_data import main as macro_main
from sql_queries.queries import latest_portfolio_metrics
from python_scripts.utils import fetch_and_process_tbill_data, prepare_data_for_simulation, calculate_cumulative_return

2025-01-12 15:25:11.252 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


Current Directory: e:\Projects\encode hackathon


In [7]:
def get_latest_model_data(base_url='http://127.0.0.1:5012'):
    url = f'{base_url}/cached-data'
    response = requests.get(url)
    data = response.json()
    print(data)
    return data

In [8]:
def data_cleaning(df,dropna=True,ffill=False):
    clean_df = clean_prices(df)
    clean_df = to_time(clean_df)
    if dropna == True:
        # clean_df = clean_df.dropna(axis=1, how='any')
        clean_df = clean_df.dropna()
    if ffill == True:
        clean_df = clean_df.resample('h').ffill().bfill()

    if '__row_index' in clean_df.columns:
        clean_df.drop(columns=['__row_index'], inplace=True)

    return clean_df

def to_time(df):
    time_cols = ['date','dt','hour','time','day','month','year','week','timestamp','date(utc)','block_timestamp']
    for col in df.columns:
        if col.lower() in time_cols and col.lower() != 'timestamp':
            df[col] = pd.to_datetime(df[col])
            df.set_index(col, inplace=True)
        elif col.lower() == 'timestamp':
            df[col] = pd.to_datetime(df[col], unit='ms')
            df.set_index(col, inplace=True)
    print(df.index)
    return df 

def clean_prices(prices_df):
    print('cleaning prices')
    # Pivot the dataframe
    prices_df = prices_df.drop_duplicates(subset=['hour', 'symbol'])
    prices_df_pivot = prices_df.pivot(
        index='hour',
        columns='symbol',
        values='price'
    )
    prices_df_pivot = prices_df_pivot.reset_index()

    # Rename the columns by combining 'symbol' with a suffix
    prices_df_pivot.columns = ['hour'] + [f'{col}_Price' for col in prices_df_pivot.columns[1:]]
    
    print(f'cleaned prices: {prices_df_pivot}')
    return prices_df_pivot

In [9]:
def network_def(chain):
        if chain == 'gnosis':
            primary_gateway = GNOSIS_GATEWAY  # Replace with your Infura URL
            backup_gateway = 'https://lb.nodies.app/v1/406d8dcc043f4cb3959ed7d6673d311a'  # Your backup gateway
        elif chain == 'arbitrum':
            primary_gateway = GATEWAY  # Replace with your Infura URL
            backup_gateway = GATEWAY
        elif chain == 'optimism':
            primary_gateway = OPTIMISM_GATEWAY  # Replace with your Infura URL
            backup_gateway = OPTIMISM_GATEWAY
        elif chain == 'ethereum':
            primary_gateway = ETHEREUM_GATEWAY  # Replace with your Infura URL
            backup_gateway = ETHEREUM_GATEWAY

        print(f'Gateway: {primary_gateway}')

        for gateway in [primary_gateway, backup_gateway]:
            w3 = Web3(Web3.HTTPProvider(gateway))
            if w3.is_connected():
                try:
                    latest_block = w3.eth.get_block('latest')['number']  # Only try this if connected
                    print(f"Connected to {chain} via {gateway}: {latest_block} block")
                    return w3, gateway
                except Exception as e:
                    print(f"Connected to {gateway} but failed to fetch latest block. Error: {e}")
            else:
                print(f"Failed to connect to {chain} via {gateway}. Trying next gateway...")

        raise ConnectionError(f"Failed to connect to {chain} network using both primary and backup gateways.")

In [10]:
def pull_data(function,path,model_name, api=False,start_date=None):
    print(f'function:{function},start_date:{start_date},path:{path},api:{api},model_name: {model_name}')

    if api:
        print(f'api True')
        # Parse dates into datetime format for consistency
        start_date = dt.datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
        
        # Use formatted date strings as needed in the dao_advisor_portfolio and lst_portfolio_prices functions
        prices = function(start_date.strftime('%Y-%m-%d %H:%M:%S'))
        
        prices_df = flipside_api_results(prices, flipside_api_key)

        prices_df.to_csv(path)
    else:
        print(f'api False')
        prices_df = pd.read_csv(path)

    dataset = {
        f'portfolio': prices_df
    }

    return dataset

In [11]:
def prices_data_func(network,
                     api_key,use_cached_data,name,days=None,
                     function=None,start_date=None,
                     backtest_period=None,filtered_assets=None):
    
    if start_date is None and backtest_period is None:
        raise KeyError("Provide either a start date or backtest_period")
    
    print(f"backtest days: {(pd.to_datetime(dt.datetime.now(dt.timezone.utc).strftime('%Y-%m-%d %H:00:00')) - pd.to_datetime(start_date)).days}")
    
    if backtest_period is None:
        backtest_period = (pd.to_datetime(dt.datetime.now(dt.timezone.utc).strftime('%Y-%m-%d %H:00:00')) - pd.to_datetime(start_date)).days * 24
        if backtest_period < 1:
            backtest_period = 1

    if function is None:

        data = token_classifier_portfolio(
            network=network,
            days=days,
            name=name,
            api_key = api_key,
            use_cached_data=use_cached_data,
            start_date = start_date,
            prices_only=True
        )

        prices_df = data_cleaning(data['portfolio'])
        prices_df
    else: 
        data = pull_data(function=function,start_date=start_date, path=f'data/{name}.csv', api=not use_cached_data,model_name=name)
        prices_df = data_cleaning(data['portfolio'])
        prices_df = prices_df[prices_df.index >= start_date].dropna()
        prices_df

    # prices_df.columns = prices_df.columns.str.replace('_Price','')
    filtered_assets_with_price = [f"{asset}_Price" for asset in filtered_assets]


    return data, prices_df[filtered_assets_with_price]

In [12]:
def update_historical_data(live_comp):
    global historical_data
    new_data = pd.DataFrame([live_comp])
    historical_data = pd.concat([historical_data, new_data]).reset_index(drop=True)
    historical_data.drop_duplicates(subset='date', keep='last', inplace=True)
    cache.set(f'historical_data', historical_data)

def update_portfolio_data(values):
    global historical_port_values
    print(f'values: {values}')
    values = pd.DataFrame([values])
    historical_port_values = pd.concat([historical_port_values, values]).reset_index(drop=True)
    historical_port_values.drop_duplicates(subset='date', keep='last', inplace=True)
    cache.set(f'historical_port_values', historical_port_values)

def update_price_data(values):
    global oracle_prices

    # Ensure the 'hour' column exists by resetting index if necessary
    if isinstance(values.index, pd.DatetimeIndex):
        values = values.reset_index().rename(columns={'index': 'hour'})
    
    if 'hour' not in values.columns:
        raise ValueError("The provided DataFrame must have a 'hour' column.")

    # Concatenate the new values with the existing oracle_prices
    oracle_prices = pd.concat([oracle_prices, values]).drop_duplicates(subset='hour', keep='last').reset_index(drop=True)
    
    # Cache the updated oracle_prices
    cache.set(f'oracle_prices', oracle_prices)

    print(f'Updated oracle_prices:\n{oracle_prices}')

def update_model_actions(actions):
    global model_actions
    print(f'model actions before update: {model_actions}')
    new_data = pd.DataFrame(actions)
    print(f'new data: {new_data}')
    model_actions = pd.concat([model_actions, new_data]).reset_index(drop=True)
    model_actions.drop_duplicates(subset='Date', keep='last', inplace=True)
    cache.set(f'actions', model_actions)

In [42]:
model_data = get_latest_model_data()

{'last_run (UTC)': '2025-01-12 20:00:00', 'model_response': {'rebalance': {'target_composition': [{'asset': 'USDC', 'reasoning': 'USDC is chosen due to its stability, important for maintaining a moderate-risk portfolio.', 'weight': 0.45}, {'asset': 'STG', 'reasoning': 'STG has shown positive recent performance, offering some growth potential.', 'weight': 0.2}, {'asset': 'CRV', 'reasoning': 'Despite recent declines, CRV has substantial market volume that might stabilize.', 'weight': 0.15}, {'asset': 'LAVA', 'reasoning': 'LAVA is maintained at a lower weight due to negative returns but keeps potential for recovery.', 'weight': 0.1}, {'asset': 'USDT', 'reasoning': 'Provides additional stability alongside USDC, balancing the higher volatility in other assets.', 'weight': 0.1}]}}, 'next_run (UTC)': 'Sun, 19 Jan 2025 20:37:37 GMT', 'prior_comp': {'ARB': 0.0, 'CRV': 0.2812656446191395, 'DAI': 0.0, 'LAVA': 0.15342397251269518, 'MAGIC': 0.0, 'STG': 4.75258353176345e-09, 'USDC': 0.56531031223594

In [14]:
next_run = model_data['next_run (UTC)']
user_message = model_data['user_message']
system_prompt = model_data['system_prompt']

In [15]:
last_run = model_data['last_run (UTC)']
last_run

'2025-01-12 20:00:00'

In [16]:
model_resonse = model_data['model_response'].get('rebalance')
model_resonse

{'target_composition': [{'asset': 'USDC',
   'reasoning': 'USDC will provide stability and acts as a safety buffer in a volatile environment, especially given the poor performance metrics of other assets.',
   'weight': 0.5},
  {'asset': 'LAVA',
   'reasoning': 'LAVA, while showing negative metrics, can retain a portion due to its existing position and potential to recover.',
   'weight': 0.25},
  {'asset': 'CRV',
   'reasoning': 'CRV has a larger market presence compared to other assets with similar poor metrics, offering potential upside with diversification.',
   'weight': 0.25}]}

In [17]:
latest_target = model_data['target_comp']
latest_target

{'ARB': 0.0,
 'CRV': 0.25,
 'DAI': 0.0,
 'LAVA': 0.25,
 'MAGIC': 0.0,
 'STG': 0.0,
 'USDC': 0.5,
 'USDT': 0.0,
 'ZRO': 0.0}

In [18]:
prior_comp = model_data['prior_comp']
prior_comp

{'ARB': 0.0,
 'CRV': 0.0,
 'DAI': 0.0,
 'LAVA': 0.11923067253569794,
 'MAGIC': 0.0,
 'STG': 0.3340539925390671,
 'USDC': 0.54671527762027,
 'USDT': 0.0,
 'ZRO': 5.730496503063168e-08}

In [19]:
network = 'arbitrum'

model = f'{network}_classifier'

params = cache.get(f'{model} params')
classifier_data = cache.get(f'{model}_portfolio')
original_prices_df = cache.get(f'{model}_prices')
days = params['days']

In [20]:
data_start_date = dt.datetime.now(dt.timezone.utc) - timedelta(days=days)
data_start_date = data_start_date.strftime('%Y-%m-%d %H:00:00')

today_utc = dt.datetime.now(dt.timezone.utc) 
formatted_today_utc = today_utc.strftime('%Y-%m-%d %H:00:00')

data_version = dt.datetime.now(dt.timezone.utc).strftime('%Y-%m-%d %H-00-00')
data_version_comp = dt.datetime.now(dt.timezone.utc).strftime('%Y-%m-%d %H:00:00') 

In [21]:
start_date = str(data_start_date)
end_date = dt.datetime.now(dt.timezone.utc).strftime('%Y-%m-%d %H:00:00') 

start_date

'2025-01-05 20:00:00'

In [22]:
filtered_assets = classifier_data['symbol'].unique()

In [23]:
data, prices_df = prices_data_func(
                            network=network, 
                            name=model,
                            api_key=flipside_api_key,
                            use_cached_data=False,
                            function=None,
                            start_date=start_date,
                            filtered_assets=filtered_assets
                            )
        
prices_df = prepare_data_for_simulation(prices_df, start_date, end_date)

backtest days: 7
use_cached_data: False
volume_threshold: 1
start_date: 2025-01-05 20:00:00
data_start_str: 2024-07-08 04:00:00
Beginning: '2025-01-05 20:00:00'


cleaning prices
cleaned prices:                          hour  ARB_Price  CRV_Price  DAI_Price  LAVA_Price  \
0    2025-01-05T20:00:00.000Z   0.894672   1.033000   1.002000    0.167994   
1    2025-01-05T21:00:00.000Z   0.902264   1.034000   0.999918    0.168680   
2    2025-01-05T22:00:00.000Z   0.898862   1.032000   1.000000    0.174682   
3    2025-01-05T23:00:00.000Z   0.911411   1.039000   1.002000    0.173347   
4    2025-01-06T00:00:00.000Z   0.916952   1.033000   0.999999    0.174486   
..                        ...        ...        ...        ...         ...   
164  2025-01-12T16:00:00.000Z   0.732592   0.834279   0.999153    0.151856   
165  2025-01-12T17:00:00.000Z   0.740609   0.841566   0.999949    0.149409   
166  2025-01-12T18:00:00.000Z   0.740173   0.841159   1.001000    0.148100   
167  2025-01-12T19:00:00.000Z   0.742616   0.839837   0.999708    0.148144   
168  2025-01-12T20:00:00.000Z   0.739661   0.828287   0.999112    0.147082   

     MAGIC_Price  STG_Price  US

e:\Projects\encode hackathon\python_scripts\utils.py:286: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  required_dates = pd.date_range(start=start_date, end=end_date, freq='H')
e:\Projects\encode hackathon\python_scripts\utils.py:293: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  price_timeseries.fillna(method='ffill', inplace=True)


In [24]:
chain = params['network']

w3, gateway = network_def(chain)

account = Account.from_key(PRIVATE_KEY)
w3.eth.default_account = account.address

Gateway: https://arb-mainnet.g.alchemy.com/v2/U23ynFAHtx1ks0s08WXlr2WR8Bs07Nsk
Connected to arbitrum via https://arb-mainnet.g.alchemy.com/v2/U23ynFAHtx1ks0s08WXlr2WR8Bs07Nsk: 294760523 block


In [25]:
portfolio = classifier_data[['symbol','token_address']]

TOKEN_CONTRACTS = {
    row['symbol']: row['token_address'] for _, row in portfolio.iterrows()
}

TOKEN_DECIMALS = get_token_decimals(TOKEN_CONTRACTS,w3)

In [26]:
prices_df.columns = [col.replace('_Price', '') for col in prices_df.columns]
prices_df.set_index('hour',inplace=True)
prices_returns = prices_df.pct_change().dropna()
prices_returns

,LAVA,MAGIC,STG,USDC,DAI,ARB,USDT,CRV,ZRO
hour,,,,,,,,,
2025-01-05 21:00:00,0.004083,-0.000298,-0.004826,0.000000,-0.002078,0.008486,-0.001493,0.000968,0.001748
2025-01-05 22:00:00,0.035582,-0.002578,0.003931,-0.000603,0.000082,-0.003771,0.000494,-0.001934,0.001745
2025-01-05 23:00:00,-0.007642,0.009862,0.018902,0.000603,0.002000,0.013961,0.000000,0.006783,0.010453
2025-01-06 00:00:00,0.006571,-0.003736,-0.007036,-0.001331,-0.001997,0.006080,-0.000797,-0.005775,0.000000
2025-01-06 01:00:00,-0.000149,-0.002993,0.026749,0.001333,-0.001084,-0.001517,0.000446,-0.003872,0.000000
...,...,...,...,...,...,...,...,...,...
2025-01-12 16:00:00,-0.012338,-0.001064,0.025052,0.000038,-0.001845,-0.000904,-0.000244,-0.003576,0.000000
2025-01-12 17:00:00,-0.016114,0.011239,0.014436,-0.000441,0.000797,0.010943,-0.000095,0.008734,0.004310
2025-01-12 18:00:00,-0.008761,-0.000626,0.048148,0.000442,0.001051,-0.000589,0.000277,-0.000484,0.000000


In [27]:
latest_prices = {
    token: float(prices_df[f"{token}"].iloc[-1])
    for token in TOKEN_CONTRACTS.keys()
    if f"{token}" in prices_df.columns
}

In [28]:
model_balances = get_balance(TOKEN_CONTRACTS,TOKEN_DECIMALS,ACCOUNT_ADDRESS,w3)

model_balances  

model_balances_usd = convert_to_usd(model_balances,latest_prices,TOKEN_CONTRACTS)
model_balances_usd

available_balance = sum(model_balances_usd.values())
available_balance

comp_dict = {
    f"{token}": balance_usd / available_balance
    for token, balance_usd in model_balances_usd.items()
}

print(f'comp_dict: {comp_dict}')

comp_dict["date"] = formatted_today_utc

update_historical_data(comp_dict)

portfolio_dict = {
    "Portfolio Value": available_balance,
    "date": formatted_today_utc
}

print(f'portfolio value: {portfolio_dict}')

update_portfolio_data(portfolio_dict)

Balances for account 0xFA475d5FB90C4b90D929a64732DA6De38a966416: {'LAVA': 0.666638, 'MAGIC': 0.0, 'STG': 6.274029548e-09, 'USDC': 0.361368, 'DAI': 0.0, 'ARB': 0.0, 'USDT': 0.0, 'CRV': 0.21701622876495752, 'ZRO': 9.073807379e-09}
balances: dict_keys(['LAVA', 'MAGIC', 'STG', 'USDC', 'DAI', 'ARB', 'USDT', 'CRV', 'ZRO'])
TOKEN_CONTRACTS.keys(): dict_keys(['LAVA', 'MAGIC', 'STG', 'USDC', 'DAI', 'ARB', 'USDT', 'CRV', 'ZRO'])
comp_dict: {'LAVA': 0.15342397251269518, 'MAGIC': 0.0, 'STG': 4.75258353176345e-09, 'USDC': 0.5653103122359494, 'DAI': 0.0, 'ARB': 0.0, 'USDT': 0.0, 'CRV': 0.2812656446191395, 'ZRO': 6.587963239417609e-08}
portfolio value: {'Portfolio Value': 0.6390816813707957, 'date': '2025-01-12 20:00:00'}
values: {'Portfolio Value': 0.6390816813707957, 'date': '2025-01-12 20:00:00'}


In [29]:
comp_df = pd.DataFrame([comp_dict]).set_index('date')

In [30]:
comp_df = comp_df.reset_index().drop(columns='date')

In [31]:
data = comp_df.iloc[0]
data = data[data != 0]  # Filter out assets with zero weight

# Convert the series to a DataFrame for plotting
df_for_pie = pd.DataFrame({'Asset': data.index, 'Weight': data.values})

In [32]:
fig1 = px.pie(
    df_for_pie,
    names='Asset',               # Column name for labels
    values='Weight',             # Column name for values
    title=f"Portfolio Composition as of {formatted_today_utc}",
    color_discrete_sequence=px.colors.qualitative.Plotly
)
fig1.show()


In [33]:
fig2 = px.scatter(
    historical_port_values,
    x='date',
    y='Portfolio Value',
    title=f"Portfolio Value ($) Time Series Through {formatted_today_utc}",
    color_discrete_sequence=px.colors.qualitative.Plotly
)

fig2.show()

In [34]:
melted = historical_data.melt(id_vars=['date'], 
                              var_name='Asset', 
                              value_name='Weight')


In [35]:
melted = melted[melted['Weight'] > 0]

In [36]:
fig3 = px.bar(
    melted,
    x='date',
    y='Weight',
    color='Asset',
    title='Portfolio Composition Over Time',
    color_discrete_sequence=px.colors.qualitative.Plotly
)

fig3.show()


In [37]:
model_resonse

{'target_composition': [{'asset': 'USDC',
   'reasoning': 'USDC will provide stability and acts as a safety buffer in a volatile environment, especially given the poor performance metrics of other assets.',
   'weight': 0.5},
  {'asset': 'LAVA',
   'reasoning': 'LAVA, while showing negative metrics, can retain a portion due to its existing position and potential to recover.',
   'weight': 0.25},
  {'asset': 'CRV',
   'reasoning': 'CRV has a larger market presence compared to other assets with similar poor metrics, offering potential upside with diversification.',
   'weight': 0.25}]}

In [38]:
last_run

'2025-01-12 20:00:00'

In [39]:
next_run

'Sun, 19 Jan 2025 20:18:48 GMT'

In [40]:
user_message

'# Instructions:\nHere are some details about my trading portfolio. Please help me make decisions to rebalance it based on the provided data.\n# Personality\nNone\n# Risk Level\nModerate risk, willing to risk some money for the right investments but not chasing every new opportunity.\nThis represents the total $USD value of the account, including positions, margin, and available funds.\n# Available Balance\n0.7347088723648053\nPortions of this \'available_balance\' can be used for placing new orders or modifying existing positions.\nAlways leave a fraction of the total \'available_balance\' as a safety buffer for unforeseen volatility.\nThe \'available_balance\' is shared by all positions, so it is important to keep track of the available value and adjust your position sizes accordingly.\n# Open Positions\n{\'LAVA\': 0.11923067253569794, \'MAGIC\': 0.0, \'STG\': 0.3340539925390671, \'USDC\': 0.54671527762027, \'DAI\': 0.0, \'ARB\': 0.0, \'USDT\': 0.0, \'CRV\': 0.0, \'ZRO\': 5.730496503

In [41]:
system_prompt

'# Instructions:\nAct as a knowledgeable cryptocurrency assistant helping users manage and optimize their trading portfolio.\nUsers understand that trading cryptocurrency is inherently risky and seek to make informed, strategic decisions to maximize their returns.\nYou will be provided with the following data:\n- `available_balance`: Represents the user\'s total available USD value for making new trades.\n- `risk_level`: Indicates the user\'s risk tolerance (e.g., Low, Moderate, High).\n- `current_positions`: A dictionary mapping cryptocurrency symbols to their current USD value holdings (e.g., {\'ETH\': 1500, \'BTC\': 1000}).\n- `market_data`: Recent performance metrics for each asset, including Sharpe Ratio, Excess Return, Latest Price, and 60-Day Return (e.g., {\'ETH\': {\'sharpe_ratio\': 1.5, \'excess_return\': 0.05, \'latest_price\': 3000, \'60d_return\': 20.0}, ...}).\n\nYour primary objective is to provide a rebalance strategy that aims to maximize the portfolio\'s returns while